## Model C2

Modifikovaný model C1

Model: vícevstupový

Data: Technical indicators, HOLCV,  Ukazatel sentimentu sentdex, informace o vyhledávání = Daily sell / daily Buy vyhledávání GT, 2007 - duben 2019

Dataset: aapl_data_2007_2019

In [0]:
from numpy import concatenate
from numpy import sqrt
from matplotlib import pyplot
from pandas import read_csv, read_excel
from pandas import read_excel
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
import keras.optimizers

In [0]:
#TO SUPERVISED
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):

	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [0]:
dataset = read_excel('/content/KO_DATASET.xlsx', header=0, index_col=0)

Column names:

'Date',	'High',	Low,	Open,	Open_mov,	'Open_bin',	'Close',	'Close_mov',	'Close_bin',	'Volume',	'Volume_mov', Volume_bin', 'Adj Close',	'rsi',	'rsi_bin',	'up_band',	'mid_band',	'low_band',	'bbp',	'bbp_bin',	'ma5',	'ma5_mov',	'ma5_bin',	'ma10',	'ma10_mov',	'ma10_bin',	'ma20',	'ema10',	'ema10_mov',	'ema_10_bin',	'ema20',	'ema20_mov',	'ema20_bin',	'macd12',	'macd26',	'macd9',	'macd',	'macd_bin',	'mom',	'mom_bin'

['Close_bin','Open_bin','Open_bin', 'bbp_bin', 'ma5_bin', 'ma10_bin', 'ema_10_bin', 'macd_bin','rsi_bin', 'bbp_bin', 'mom_bin', 'AAPL_GT', 'AAPL_SENT']]

In [0]:
#Určit, se kterými proměnnými bude model pracovat, ['Close','Volume','rsi','ma5','ma10','ma20','ema10','ema20','macd','mom']

dataset = dataset[['Close_bin','Open_bin','Open_bin', 'bbp_bin', 'ma5_bin', 'ma10_bin', 'ema_10_bin', 'macd_bin','rsi_bin', 'bbp_bin', 'mom_bin', 'KO_GT', 'KO_SENT']]
values = dataset.values

In [0]:
# ensure all data is float
values = values.astype('float32')


In [0]:
# frame as supervised learning
#změněno na values ze scaled
reframed = series_to_supervised(values, 10, 2)
reframed = DataFrame(reframed)

Dropping columns not used for y output

Dropnout všechny +1, kromě var4(t+1)

reframed = reframed.drop(['var1(t+1)', 'var2(t+1)', 'var3(t+1)',
                          'var5(t+1)', 'var6(t+1)', 'var7(t+1)',
                          'var8(t+1)', 'var9(t+1)', 'var10(t+1)',
                          'var11(t+1)', 'var12(t+1)', 'var13(t+1)',
                          'var14(t+1)', 'var15(t+1)', 'var16(t+1)',
                         'var17(t+1)', 'var18(t+1)', 'var19(t+1)',
                          'var20(t+1)'], axis=1)
                          
                          'var5(t+1)', 'var6(t+1)', 'var7(t+1)',
                          'var8(t+1)', 'var9(t+1)', 'var10(t+1)',  'var11(t+1)'

In [0]:
#Upravit, dropnout správné
reframed = reframed.drop(['var4(t+1)', 'var2(t+1)', 'var3(t+1)',
                          'var5(t+1)', 'var6(t+1)', 'var7(t+1)','var8(t+1)','var9(t+1)', 'var10(t+1)', 'var11(t+1)','var12(t+1)','var13(t+1)'], axis =  1) 

In [0]:
#Časy: 1.1.2008 - 252, konec 2015 - 2266, konec 2017 - 2769 - při 60
# split into train and test sets


#Čas: 246: 1.1.2008, 2514 - konec 2016


train = reframed.values[31:1041, :]
validation = reframed.values[1041:1290, :]
test = reframed.values[1290:, :]

In [0]:
validation.shape

In [0]:
#Train/test - ale je to vlastně train, validation.
train_X, train_y = train[:, :-1], train[:, -1]
validation_X, validation_y = validation[:, :-1], validation[:, -1]




In [0]:
# reshape input to be 3D [samples, timesteps, features]
# Tuhle část upravit, ověřit, jetsli to dává smysl
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
validation_X = validation_X.reshape((validation_X.shape[0], 1, validation_X.shape[1]))
print(train_X.shape, train_y.shape, validation_X.shape, validation_y.shape)

In [0]:
model = Sequential()
model.add(LSTM(100, return_sequences=True,input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.4))
# Fourth LSTM layer
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

model.summary()

Fiffing the model - trénování modelu pomocí trénovacích dat, validace pomocí validačních

In [0]:
#Trénování modelu
history = model.fit(train_X, train_y, epochs=7, batch_size=10, validation_data=(validation_X, validation_y), verbose=2, shuffle=False)

In [0]:
pyplot.figure(figsize=(10,6))
pyplot.plot(history.history['acc'], label='Train set', color = '#18572D')
pyplot.plot(history.history['val_acc'], label='Validation set', color = '#FF9000')
pyplot.legend()
pyplot.xlabel('Epochs')
pyplot.ylabel('KO Test/Validation Accuracy')
pyplot.show()

In [0]:
pyplot.figure(figsize=(10,6))
pyplot.plot(history.history['loss'], label='Train set', color = '#18572D')
pyplot.plot(history.history['val_loss'], label='Validation set', color = '#FF9000')
pyplot.legend()
pyplot.xlabel('Epochs')
pyplot.ylabel('KO Test/Validation Loss')
pyplot.show()

In [0]:
#Ohodnocení pomocí accuracy, loss
scores = model.evaluate(validation_X, validation_y, verbose=2)
print("%.6f" % (scores[1]))
print("%.6f" % (scores[0]))


Predikce na validačních datech

In [0]:
yhat_validation = model.predict(validation_X)
yhat_validation

### Predikce na nových datech

In [0]:
test_X = test[:, :-1]
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

print(test_X.shape)

yhat = model.predict(test_X)

In [0]:
yhat.shape

### Převedení výsledku do Excel, aby se s ním dalo nadále pracovat

konečná přesnost je určena pomocí modelu v excelu, kde se srovnávají hodnoty reálné a binární hodnoty predikované- tedy půjde-li cena vzhůru či klesne-li

In [0]:
Vysledek = DataFrame(yhat)
Vysledek.to_excel('/content/KO_C22_Vysledek.xlsx')